[![image-1.png](https://i.postimg.cc/Vk3VrVXM/image-1.png)](https://postimg.cc/v4tzCXqH)

<div style="padding:15px 20px 20px 20px;
            color:#004346;
            font-size:40px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">
<p style="font-weight: bold; text-align: center;">Прогнозирование оттока клиентов</p>

</div>

<div style="padding:0px 40px 30px;
            color:#004346;
            font-size:110%;
            display:fill;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:450;">
    
__Заказчик:__ Оператор связи «ТелеДом».
    
__Описание задачи:__ Для борьбы с оттоком клиентов, сотрудники компании будут предлагать промокоды и специальные условия тем абонентам, которые имеют намерение прекратить пользоваться услугами связи. Это позволит заранее выявлять пользователей, склонных к расторжению договора.

__Постановка задачи:__ Разработать модель для прогнозирования вероятности расторжения договора абонентом.
    
__Оценка качества модели:__ Метрики ROC-AUC, значение должно быть не менее 0.85. 
    
__Описание данных:__ персональные данные о некоторых клиентах, информация об их тарифах и услугах.
 
Данные хранятся в базе данных PostgreSQL. Она состоит из нескольких таблиц:
    
- `contract` — информация о договорах;
- `personal` — персональные данные клиентов;
- `internet` — информация об интернет-услугах;
- `phone` — информация об услугах телефонии.
    
    
    
- файл `train_dataset.csv` содержит информацию, необходимую для обучения:
    - имя файла изображения;
    - идентификатор описания в формате: <имя файла изображения>#<порядковый номер описания>;
    - текст описания (доступно до 5 описаний)
    
- директория `/train_images/` содержит изображения для тренировки моделей.
    
- файл `CrowdAnnotations.tsv` включает данные по соответствию изображения и описания, полученные с помощью краудсорсинга:
    - имя файла изображения;
    - идентификатор описания;
    - доля людей, подтвердивших, что описание соответствует изображению;
    - количество человек, подтвердивших, что описание соответствует изображению;
    - количество человек, подтвердивших, что описание не соответствует изображению.

- файл `ExpertAnnotations.tsv` включает по соответствию изображения и описания, полученные в результате опроса экспертов:
    - имя файла изображения;
    - идентификатор описания;
    - оценки трёх экспертов по шкале от 1 до 4:
         - 1 — изображение и запрос совершенно не соответствуют друг другу,
         - 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует,
         - 3 — запрос и текст соответствуют с точностью до некоторых деталей,
         - 4 — запрос и текст соответствуют полностью.
    
- файл `test_queries.csv` содержит информацию, необходимую для тестирования:
    - идентификатор запроса;
    - текст запроса;
    - релевантное изображение.
    
- директория `/test_images/` содержит изображения для тестирования моделей.

__Юридические ограничения:__ Поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно, текстов, изображений, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16-ти лет.

</div>    